In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel
import ast
from google.colab import files
uploaded = files.upload()

Saving movies.csv to movies.csv


In [ ]:
movies = pd.read_csv('movies.csv', low_memory=False)

In [ ]:
print(movies.head())

   Unnamed: 0      id    imdb_id  popularity       budget       revenue  \
0           0  135397  tt0369610   32.985763  150000000.0  1.513529e+09   
1           1   76341  tt1392190   28.419936  150000000.0  3.784364e+08   
2           2  262500  tt2908446   13.112507  110000000.0  2.952382e+08   
3           3  140607  tt2488496   11.173104  200000000.0  2.068178e+09   
4           4  168259  tt2820852    9.335014  190000000.0  1.506249e+09   

                 original_title  \
0                Jurassic World   
1            Mad Max: Fury Road   
2                     Insurgent   
3  Star Wars: The Force Awakens   
4                     Furious 7   

                                                cast  \
0  Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...   
1  Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...   
2  Shailene Woodley|Theo James|Kate Winslet|Ansel...   
3  Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...   
4  Vin Diesel|Paul Walker|Jason Statham|Michelle ...   

 

In [ ]:
print(movies['genres'].head())

0    Action|Adventure|Science Fiction|Thriller
1    Action|Adventure|Science Fiction|Thriller
2           Adventure|Science Fiction|Thriller
3     Action|Adventure|Science Fiction|Fantasy
4                        Action|Crime|Thriller
Name: genres, dtype: object


In [ ]:
movies = movies.dropna(subset=['overview', 'genres'])

In [ ]:
def convert_genres(genres_str):
    try:
        genres = ast.literal_eval(genres_str)
        genre_list = [genre['name'] for genre in genres]
        return " ".join(genre_list)
    except (ValueError, SyntaxError) as e:
        return ""

In [ ]:
movies['genres'] = movies['genres'].apply(convert_genres)
movies['overview'] = movies['overview'].fillna('')
movies['combined_features'] = movies['overview'] + " " + movies['genres']


In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(movies['combined_features'])

In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
movie_indices = pd.Series(movies.index, index=movies['original_title']).drop_duplicates()

In [ ]:
def get_recommendations(original_title, cosine_sim=cosine_sim):
    idx = movie_indices[original_title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices_recommend = [i[0] for i in sim_scores]
    return movies['original_title'].iloc[movie_indices_recommend]

In [ ]:
recommended_movies = get_recommendations('The Godfather')
print("Recommended Movies:")
print(recommended_movies)

Recommended Movies:
1267                            The Godfather: Part III
835                                      Mambo Italiano
831                               House of 1000 Corpses
1139                                 Death at a Funeral
932                                       Four Brothers
663                                             Rampart
791     For Greater Glory - The True Story of Cristiada
337                                 The Next Three Days
981                                        The Fountain
953                                               Brick
Name: original_title, dtype: object
